# How to create a CC-U gate given CNOT and unary gates

We look for such $V$, that $V^2=U$. How?

For rotations this is easy: $Rot(2\alpha)=Rot^2(\alpha)$.

For reflections this can be a problem. Still, we have [Matrix square root](https://en.wikipedia.org/wiki/Square_root_of_a_matrix).

### How to?

Note, that if we apply $V$, and then we can branch into $V$ again, or $V^\dagger$. First path will bring us to $U$, whereas the second will cancel the operation, as $V^\dagger = V^{-1}$. So, using this idea we can build a $CCU$!

![](https://qiskit.org/textbook/ch-gates/images/iden2.png) [pic](https://qiskit.org/textbook/ch-gates/more-circuit-identities.html)

### Go ahead and practice!

Implement CCZ.

$CCZ$ matrix is:

In [2]:
import numpy as np
CCZ = np.eye(8)
CCZ[-1, -1] = -1
print(CCZ)

[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.]]


What we know, is that $S=\sqrt{Z}$.

$S=\begin{pmatrix}1 & 0 \\ 0 & e^{\frac{i\pi}{2}}\end{pmatrix}=\begin{pmatrix}1 & 0 \\ 0 & i\end{pmatrix}$

What we also know according to [Barenco et al.](https://arxiv.org/pdf/quant-ph/9503016.pdf) is that _"For any unitary 2×2 matrix U, a CC-U gate can be simulated by at most 16 basic gates: eight 1-bit gates and eight CNOT gates"_.

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
import matplotlib.pyplot as plt

qr = QuantumRegister(3, 'qbits')
cr = ClassicalRegister(3, 'bits')
qc = QuantumCircuit(qr, cr)


from qiskit.circuit.library.standard_gates import SGate

# this is a construction of random controlled gate
CS = SGate().control(1)

# this is how we can inverse
CSI = SGate().inverse().control(1)

qc.append(CS, [qr[1], qr[2]])
# TODO COMPLETE THE CIRCUIT!!!!


qc.measure(qr, cr)
qc.draw(output='mpl')
plt.show()

print(qc.qasm())

Run this on QPU:

In [ ]:
from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider('ibm-q')

available_cloud_backends = provider.backends()

queue = []
for backend in available_cloud_backends:
    status = backend.status()
    is_operational = status.operational
    jobs_in_queue = status.pending_jobs
    if not is_operational: 
        continue
    if 'simulator' in backend.name():
        continue
    # print(f"{backend} with {backend.configuration().n_qubits} qubits is online with a queue={jobs_in_queue}")
    queue.append((backend.configuration().n_qubits, -jobs_in_queue, backend.name()))

queue.sort()
qpu = provider.get_backend(queue[-1][-1])
job = execute(qc, backend=qpu)
from qiskit.tools.monitor import job_monitor
job_monitor(job)

print(job.result())

In [24]:
dir(job)
job.status().value

'job incurred error'